# Greedy Technique

Given an array of size n that contain either ‘G’rab car or ‘P’assenger. 

Each Grab car can pick up only one passenger. And each Grab car cannot pick up a passenger who is more than K units away from Grab car.

1. Write a program using Brute force approach to find number of all solutions 
    that give the maximum number of Passenger(s) that can ride Grab(s).
2. Write a program using Greedy Technique to find a solution that gives the maximum number of Passengers(s) that can ride Grab(s).

For example, if an array consists of {‘G’, ‘P’, ‘P’, ‘G’, ‘P’} and we set k = 1, 

then the output the maximum number passenger can ride Grab would be 2. 

The first Grab picks up the first passenger and the second Grab picks up either the second or third passenger.

---

## Reading test case

In [115]:
def read_input(filename):
    with open(filename, 'r') as f:
        arr = list(f.readline().strip())
        k = int(f.readline().strip())
    return arr, k

# The functions 'can_pickup', 'recursive_search' and 'find_max_pickups' remain unchanged from the previous answer

# Reading the input
filename = 'Example_LAB_3.txt'
arr, k = read_input(filename)
print(f"arr = {arr}\nk   = {k}")

arr = ['P', 'P', 'G', 'G', 'G', 'P', 'G', 'P', 'G', 'P', 'G', 'P', 'P', 'P', 'P', 'G']
k   = 4


### Brute force

In [105]:
def can_pickup(g_index, p_index, k):
    """Checks if a Grab car at g_index can pick up a passenger at p_index"""
    return abs(g_index - p_index) <= k

def brute_force(arr, k, start=0, pickups=[]):
    """Recursively find all possible pairings of Grab cars and passengers"""
    n = len(arr)
    max_pickups = len(pickups)
    best_solutions = [pickups]

    for i in range(start, n):
        if arr[i] == 'G':
            for j in range(n):
                if arr[j] == 'P' and not any(p[1] == j for p in pickups) and can_pickup(i, j, k):
                    new_pickups, new_solutions = brute_force(arr, k, i + 1, pickups + [(i, j)])
                    if new_pickups == max_pickups:
                        best_solutions.extend(new_solutions)
                    elif new_pickups > max_pickups:
                        max_pickups = new_pickups
                        best_solutions = new_solutions

    return max_pickups, best_solutions

max_passengers, solutions = brute_force(arr, k)

print(f"arr = {arr}\nk   = {k}")
print(f"Maximum passengers: {max_passengers}")
print(f"all possible solutions: {len(solutions)}")

arr = ['P', 'P', 'G', 'G', 'G', 'P', 'G', 'P', 'G', 'P', 'G', 'P', 'P', 'P', 'P', 'G']
k   = 4
Maximum passengers: 7
all possible solutions: 672


In [106]:
def calculate_distance(solution):
    """Calculate the total distance for a given solution."""
    return sum(abs(g - p) for g, p in solution)

def get_solutions_for_distance(solutions, distances, target_distance):
    """Get solutions corresponding to a specific distance."""
    return [solution for solution, distance in zip(solutions, distances) if distance == target_distance]

def print_solutions(title, solutions_list, distance=None):
    """Print a list of solutions with a title."""
    if distance is not None:
        print(f"{title} Distance: {distance}")
    for solution in solutions_list:
        print(solution)
    print("\n" + "-"*50 + "\n")

# Calculate distances for each solution
distances = [calculate_distance(solution) for solution in solutions]

# Find minimum and maximum distances
min_distance = min(distances)
max_distance = max(distances)

# Extract solutions with minimum and maximum distances
solutions_with_min_distance = get_solutions_for_distance(solutions, distances, min_distance)
solutions_with_max_distance = get_solutions_for_distance(solutions, distances, max_distance)

# Print all solutions with their distances
for idx, (solution, distance) in enumerate(zip(solutions, distances), 1):
    print(f"Solution {idx}: {solution}, Total Distance: {distance}")

print("\n" + "-"*50 + "\n")

# Print solutions with minimum and maximum distances
print_solutions("Minimum", solutions_with_min_distance, min_distance)
print_solutions("Maximum", solutions_with_max_distance, max_distance)

Solution 1: [(2, 0), (3, 1), (4, 5), (6, 7), (8, 9), (10, 11), (15, 12)], Total Distance: 11
Solution 2: [(2, 0), (3, 1), (4, 5), (6, 7), (8, 9), (10, 11), (15, 13)], Total Distance: 10
Solution 3: [(2, 0), (3, 1), (4, 5), (6, 7), (8, 9), (10, 11), (15, 14)], Total Distance: 9
Solution 4: [(2, 0), (3, 1), (4, 5), (6, 7), (8, 9), (10, 12), (15, 11)], Total Distance: 13
Solution 5: [(2, 0), (3, 1), (4, 5), (6, 7), (8, 9), (10, 12), (15, 13)], Total Distance: 11
Solution 6: [(2, 0), (3, 1), (4, 5), (6, 7), (8, 9), (10, 12), (15, 14)], Total Distance: 10
Solution 7: [(2, 0), (3, 1), (4, 5), (6, 7), (8, 9), (10, 13), (15, 11)], Total Distance: 14
Solution 8: [(2, 0), (3, 1), (4, 5), (6, 7), (8, 9), (10, 13), (15, 12)], Total Distance: 13
Solution 9: [(2, 0), (3, 1), (4, 5), (6, 7), (8, 9), (10, 13), (15, 14)], Total Distance: 11
Solution 10: [(2, 0), (3, 1), (4, 5), (6, 7), (8, 9), (10, 14), (15, 11)], Total Distance: 15
Solution 11: [(2, 0), (3, 1), (4, 5), (6, 7), (8, 9), (10, 14), (15, 1

---

### Greedy

In [116]:
def greedy_max_passenger(arr, k):
    n = len(arr)
    paired = [False] * n  # To mark passengers that have been paired
    pairs = []

    # Count number of G's within k distance for each P
    p_count = [0] * n
    for i in range(n):
        if arr[i] == 'P':
            for j in range(max(0, i-k), min(n, i+k+1)):
                if arr[j] == 'G':
                    p_count[i] += 1

    for i in range(n):
        if arr[i] == 'G':
            # Prioritize P with fewest G's around; if tie, pick nearest P
            min_count = float('inf')
            chosen_p = None
            for j in range(max(0, i-k), min(n, i+k+1)):
                if arr[j] == 'P' and not paired[j] and p_count[j] < min_count:
                    min_count = p_count[j]
                    chosen_p = j

            if chosen_p is not None:
                pairs.append((i, chosen_p))
                paired[chosen_p] = True

    return len(pairs), pairs

max_passengers, solution = greedy_max_passenger(arr, k)

print(arr)
print(f"Maximum passengers: {max_passengers}")
print(f"Solution: {solution}")
distances = [calculate_distance(solution) for solution in solutions]
print(f"Total distance: {calculate_distance(solution)}")


['P', 'P', 'G', 'G', 'G', 'P', 'G', 'P', 'G', 'P', 'G', 'P', 'P', 'P', 'P', 'G']
Maximum passengers: 7
Solution: [(2, 0), (3, 1), (4, 5), (6, 9), (8, 11), (10, 13), (15, 14)]
Total distance: 15


----